# 任务目标--聚合file_id，index，统计api，tid

# 调包区


In [1]:
from tqdm.notebook import tqdm
import random
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import gc

# 警告忽略
import warnings
warnings.filterwarnings("ignore")

# matplotlib字体设置
plt.rcParams["font.family"] = "Songti SC"
plt.rcParams["axes.unicode_minus"] = False

# matplotlib警告忽略
pd.plotting.register_matplotlib_converters()


# 观看Dataframe长度
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
# 浮点数位长度
pd.set_option('display.precision',5)

# 显示多个结果
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # ['all', 'last', 'last_expr', 'none', 'last_expr_or_assign']

In [2]:
from sklearn.preprocessing import StandardScaler,LabelEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,KFold
from sklearn.naive_bayes import BernoulliNB,GaussianNB,MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from lightgbm.sklearn import LGBMClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,roc_auc_score,auc,f1_score
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_squared_log_error
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import keras
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D , GlobalAveragePooling1D,MaxPool1D
from keras import Model
from keras.models import Sequential
from keras.optimizers import Adam,SGD # 优化方法
from keras.callbacks import EarlyStopping,ModelCheckpoint,RemoteMonitor,CSVLogger
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [4]:
import networkx as nx
import node2vec
from node2vec import Node2Vec
from gensim.models import word2vec

# 打开文件

In [5]:
path = './sampledata' # 打开样本数据
#path = './fulldata' # 打开全量数据

train_data = pd.read_csv(f'{path}/security_train.csv')
test_data = pd.read_csv(f'{path}/security_test.csv')

test_data['label']=np.nan

# 设置版本号
version = 'v1.3.3'

# 特征工程

## 生成特征文档

In [6]:
basic_feature = ['file_id','label','api','tid','index']

train_fe = train_data[basic_feature].drop_duplicates(['file_id','label'])
test_fe = test_data[basic_feature].drop_duplicates(['file_id','label'])


## 构建一阶统计值特征（count,nunique,min、max、sum、mean、median）

In [7]:
# 得到统计值特征（count,nunique,min、max、sum、mean、median）
def creat_stats_feature(tem_df):
    # 以file_id 为单位聚合，统计每一个file_id下的情况
    group_df = tem_df.groupby('file_id')

    # 得到 count,nunique,min、max、sum、mean、median的统计值
    stats_dict = {
        "api":['count', 'nunique'],
        "tid":['count', 'nunique', 'min', 'max', 'mean', 'median','std'], 
        "index":['count', 'nunique', 'min', 'max', 'mean', 'median','std'],}

    stats_df = group_df.agg(stats_dict)
    stats_df.columns = ["file_"+"_".join(tup) for tup in stats_df.columns]
    stats_df = stats_df.reset_index()
    return(stats_df)



# 构建训练集的特征
stats_df = creat_stats_feature(train_data)
train_fe = train_fe.merge(stats_df,on='file_id',how='left')

# 构建测试集的特征
stats_df = creat_stats_feature(test_data)
test_fe = test_fe.merge(stats_df,on='file_id',how='left')


# 回收内存
del stats_df
gc.collect()


3

## 构建二阶段特征 -- 聚合file_id，api，统计tid，index

In [8]:
def creat_stats_feature2(tem_df):
    stats_parmas = {'tid':['count','nunique'],
                    'index':['count','nunique','max','min','median','std'],
                   }
    tem_df = tem_df.groupby(['file_id','api']).agg(stats_parmas)
    tem_df.columns = [f'file_api_{i}_{b}' for i,b in tem_df.columns]

    df_list = []
    for columns in tem_df.columns:
        pivot_df = pd.pivot_table(data=tem_df,index='file_id',columns='api',values=columns,fill_value=0)
        pivot_df.columns = [f'{columns}_{feature}' for feature in pivot_df.columns]
        df_list.append(pivot_df)

    pivot_df = pd.concat(df_list,axis=1)

    return pivot_df



# 构建训练集的特征
pivot_df = creat_stats_feature2(train_data)
train_fe = train_fe.merge(pivot_df,on='file_id',how='left')

# 构建测试集的特征
pivot_df = creat_stats_feature2(test_data)
test_fe = test_fe.merge(pivot_df,on='file_id',how='left')


# 回收内存
del pivot_df

## 构建二阶特征 -- 聚合file_id，index，统计api

In [9]:
def creat_stats_feature3(tem_df):
    stats_parmas = {'api':['count','nunique'],
                    'tid':['nunique','max','min','median','std'],
                   }
    tem_df = tem_df.groupby(['file_id','index']).agg(stats_parmas)
    tem_df.columns = [f'file_index_{i}_{b}' for i,b in tem_df.columns]
    stats_df = tem_df.fillna(0)
    return stats_df

# 构建训练集的特征
stats_df = creat_stats_feature3(train_data)
train_fe = train_fe.merge(stats_df,on=['file_id','index'],how='left')

# 构建测试集的特征
stats_df = creat_stats_feature3(test_data)
test_fe = test_fe.merge(stats_df,on=['file_id','index'],how='left')


# 回收内存
del stats_df

## 构建三阶特征 -- 聚合file_id，api，tid 统计index

In [10]:
## 构建三阶特征 -- 聚合file_id，api，tid 统计index(nunique,max,min,median,std)
def creat_stats_feature4(tem_df):
    stats_parmas = {'index':['nunique','max','min','median','std'],
                   }
    tem_df = tem_df.groupby(['file_id','api','tid']).agg(stats_parmas)
    tem_df.columns = [f'file_api_tid_{i}_{b}' for i,b in tem_df.columns]
    stats_df = tem_df.fillna(0)
    return stats_df


# 构建训练集的特征
stats_df = creat_stats_feature4(train_data)
train_fe = train_fe.merge(stats_df,on=['file_id','api','tid'],how='left')

# 构建测试集的特征
stats_df = creat_stats_feature4(test_data)
test_fe = test_fe.merge(stats_df,on=['file_id','api','tid'],how='left')


# 回收内存
del stats_df

## 构建三阶特征 -- 聚合file_id，tid，api 统计index

In [11]:
## 构建三阶特征 -- 聚合file_id，tid，api 统计index(nunique,max,min,median,std)
def creat_stats_feature5(tem_df):

    stats_parmas = {'index':['nunique','max','min','median','std'],
                   }
    tem_df = tem_df.groupby(['file_id','tid','api']).agg(stats_parmas)
    tem_df.columns = [f'file_tid_api_{i}_{b}' for i,b in tem_df.columns]
    stats_df = tem_df.fillna(0)
    return stats_df


# 构建训练集的特征
stats_df = creat_stats_feature5(train_data)
train_fe = train_fe.merge(stats_df,on=['file_id','tid','api'],how='left')

# 构建测试集的特征
stats_df = creat_stats_feature5(test_data)
test_fe = test_fe.merge(stats_df,on=['file_id','tid','api'],how='left')


# 回收内存
del stats_df

## 补全columns

In [12]:
fe_data = pd.concat([train_fe,test_fe],axis=0)
train_fe = fe_data[fe_data['label'].notnull()].fillna(0)
test_fe = fe_data[fe_data['label'].isnull()].fillna(0)                                      

del fe_data
gc.collect()

train_fe.shape
test_fe.shape

0

(278, 2073)

(260, 2073)

## 删除无用特征

In [13]:
del_list = ['api']
train_fe = train_fe.drop(del_list,axis=1)
test_fe = test_fe.drop(del_list,axis=1)

# 特征工程保存

In [14]:
path = "./temdata"
train_fe.to_pickle(f'{path}/train_fe_{version}.pkl')
test_fe.to_pickle(f'{path}/test_fe_{version}.pkl')

In [15]:
path = "./temdata"
train_fe = pd.read_pickle(f'{path}/train_fe_{version}.pkl')
test_fe = pd.read_pickle(f'{path}/test_fe_{version}.pkl')

# 数据切分

In [16]:
# 数据划分
raw_x,raw_y = train_fe.drop('label',axis=1),train_fe['label']
sub_x,sub_y = test_fe.drop('label',axis=1),test_fe['label']

# 进行数据切分
train_x,vaild_x,train_y,vaild_y = train_test_split(raw_x,raw_y,test_size=0.2)

# 建模尝试

## 使用lgb进行建模预测

In [17]:
# 参数设置
gpu=False

#############################################

if gpu==True:
    params = {
    'device': 'gpu',
    'gpu_platform_id': 0,
    'gpu_device_id': 0}
else:
    params = {}




#############################################

model = lgb.LGBMClassifier(
            num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='multiclass',
            max_depth=-1, learning_rate=0.005, min_child_samples=3, random_state=2021,
            n_estimators=2000, subsample=1, colsample_bytree=1,**params)


#############################################
# 模型训练
# model.fit(
#     train_x, train_y,
#     eval_metric='logloss', eval_set=[(train_x, train_y), (vaild_x, vaild_y)],
#     verbose=100,
#     #早停法，如果auc在10epoch没有进步就stop
#     early_stopping_rounds=1000 )

model.fit(raw_x,raw_y)


# 预测
pred_prob = model.predict_proba(sub_x)


#############################################
# 转化为提交文件
# 转化为DataFrame
sub_df = pd.DataFrame(pred_prob)
sub_df.insert(0,"file_id",sub_x['file_id'].reset_index(drop=True))

# 更改列名
sub_df.columns = list(map(lambda x:f"prob{x}" if x!='file_id' else x,sub_df.columns))
sub_df = sub_df.astype('double')
sub_df['file_id'] = sub_df['file_id'].astype(int)

# 保存在本地
path = './outdata'
sub_df.to_csv(f'{path}/lgb_{version}.csv',index=None)


#############################################
# 观察特征重要性
feature_list = train_x.columns
value = model.feature_importances_.flatten()
print(pd.Series(value,index=feature_list).sort_values(ascending=False).to_string())


#############################################
# 保存模型
import joblib
# 模型存储
joblib.dump(model, f'./modelfile/lgb_{version}.pkl')
# 模型加载
# model = joblib.load('lgb.pkl')
print('模型保存完毕！')

LGBMClassifier(colsample_bytree=1, learning_rate=0.005, min_child_samples=3,
               n_estimators=2000, objective='multiclass', random_state=2021,
               reg_alpha=0.25, reg_lambda=0.25, subsample=1)

file_api_index_min_NtClose                                  1700
file_index_tid_nunique                                      1262
file_api_index_min_LdrGetProcedureAddress                   1232
file_api_index_min_LdrLoadDll                               1232
file_api_index_min_LdrGetDllHandle                          1123
file_tid_median                                             1112
tid                                                         1107
file_api_index_median_LdrGetProcedureAddress                1036
file_api_tid_count_CreateServiceA                            901
file_api_index_min_NtCreateFile                              863
file_id                                                      855
file_api_index_std_LdrUnloadDll                              796
file_api_index_std_SetStdHandle                              757
file_api_tid_count_NtTerminateProcess                        749
file_index_tid_std                                           718
file_api_index_min_RegClo

['./modelfile/lgb_v1.3.3.pkl']

模型保存完毕！


## 使用xgb进行建模预测

In [18]:
# 参数设置
gpu=False

#############################################

if gpu==True:
    params = {'tree_method': 'gpu_hist'}

else:
    params = {}



#############################################

model = xgb.XGBClassifier(
            max_depth=9, learning_rate=0.005, n_estimators=2000, 
            objective='multi:softprob', 
            subsample=0.8, colsample_bytree=0.8, 
            min_child_samples=3, eval_metric='mlogloss', reg_lambda=0.5,**params)


#############################################
# 模型训练
# model.fit(
#     train_x, train_y,
#     eval_metric='mlogloss', eval_set=[(train_x, train_y), (vaild_x, vaild_y)],
#     verbose=100,
#     #早停法，如果auc在10epoch没有进步就stop
#     early_stopping_rounds=1000 )


model.fit(raw_x,raw_y)

# 预测
pred_prob = model.predict_proba(sub_x)


#############################################
# 转化为提交文件
# 转化为DataFrame
sub_df = pd.DataFrame(pred_prob)
sub_df.insert(0,"file_id",sub_x['file_id'].reset_index(drop=True))

# 更改列名
sub_df.columns = list(map(lambda x:f"prob{x}" if x!='file_id' else x,sub_df.columns))

# 保存在本地
path = './outdata'
sub_df = sub_df.astype('double')
sub_df['file_id'] = sub_df['file_id'].astype(int)
sub_df.to_csv(f'{path}/xgb_{version}.csv',index=None)


#############################################
# 观察特征重要性
feature_list = train_x.columns
value = model.feature_importances_.flatten()
print(pd.Series(value,index=feature_list).sort_values(ascending=False).to_string())


#############################################
# 保存模型
import joblib
# 模型存储
joblib.dump(model, f'./modelfile/xgb_{version}.pkl')
# 模型加载
# model = joblib.load('xgb.pkl')
print('模型保存完毕！')

[23:09:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eval_metric='mlogloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.005, max_delta_step=0,
              max_depth=9, min_child_samples=3, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=2000, n_jobs=0,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=0.5, scale_pos_weight=None, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

file_api_index_max_Thread32First                            4.58674e-02
file_api_index_max_InternetCloseHandle                      9.65775e-03
file_api_tid_nunique_FindResourceExA                        8.83381e-03
file_api_index_count_Thread32Next                           7.77746e-03
file_api_index_count_RtlRemoveVectoredExceptionHandler      7.06547e-03
file_api_index_nunique_SetStdHandle                         6.63137e-03
file_index_tid_nunique                                      6.20889e-03
file_api_tid_nunique_RtlRemoveVectoredExceptionHandler      6.15101e-03
file_api_index_nunique_DrawTextExW                          6.10801e-03
file_api_tid_count_FindResourceExA                          6.02690e-03
file_api_tid_count_RtlRemoveVectoredExceptionHandler        5.94579e-03
file_api_index_count_DrawTextExW                            5.80565e-03
file_api_index_std_GetFileSizeEx                            5.39571e-03
file_api_index_count_SetStdHandle                           5.27

['./modelfile/xgb_v1.3.3.pkl']

模型保存完毕！


# 进行模型融合

## 普通的加权融合

In [19]:
# 打开之前的预测结果proba
path = './outdata'
lgb_pred = pd.read_csv(f'{path}/lgb_{version}.csv').set_index('file_id')
xgb_pred = pd.read_csv(f'{path}/xgb_{version}.csv').set_index('file_id')

# 设置权重
lgb_weight = 0.4
xgb_weight = 0.6

# 进行加权融合
mix_pred = (lgb_weight*lgb_pred)+(xgb_weight*xgb_pred)

# 保存加权融合后的结果
mix_pred.to_csv(f'{path}/mix_{version}.csv')